# M1

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.visualization.modules import CanvasGrid
from Tumor_cells import Tumor_cells
from mesa.visualization.ModularVisualization import ModularServer

In [ ]:
"""
    Represents an M1 macrophage agent in the model.

    Attributes:
        position (tuple): The (x, y) position of the agent in the grid.
        killing_capacity (int): The number of tumor cells the agent can kill.
        prob_kill (float): Probability of killing a tumor cell in a step.
        prob_migrate (float): Probability of moving to a new position in a step.
        prob_death (float): Probability of dying in a step.
        alive (bool): Indicates whether the agent is alive.
"""
class M1(Agent):
    
    """
        Initializes an M1 macrophage agent.

        Args:
            agent_id (int): Unique identifier for the agent.
            position (tuple): Initial position of the agent in the grid.
            model (Model): The model the agent belongs to.
    """


## Initialization
Initializes an M1 macrophage agent.
Sets up the M1 macrophage with its unique ID, position, and initial attributes such as killing capacity, probabilities of actions, and alive status.
Initialization ensures each M1 macrophage starts with consistent properties and is ready to perform its role in the simulation.


In [ ]:

    def __init__(self, agent_id, position, model):
        super().__init__(agent_id, model)
        self.position = position
        self.killing_capacity = 11       # Killing capacity 
        self.prob_kill = 0.0306          # Probability of killing
        self.prob_migrate = 0.2667       # Probability of migration
        self.prob_death = 0.0049         # Probability of death
        self.alive = True


## Eat
Consumes a specified amount of nutrition from the environment.
Allows the M1 macrophage to interact with its environment by consuming resources, which might affect its ability to function.
Simulating resource consumption models the dependency of macrophages on their environment for energy and survival.


In [ ]:

    def eat(self, val):
        self.model.eat_nutrition(val)
    
        """
        Executes one step of the agent's behavior:
        - Checks if the agent dies based on `prob_death`.
        - Migrates to a neighboring cell with `prob_migrate`.
        - Attempts to kill a tumor cell in its neighborhood with `prob_kill`.
        """


## Step
Executes one step of the M1 macrophage agent's behavior.
Defines the agent's lifecycle at each step, including:
- **Death:** The agent may die based on its `prob_death`.
- **Migration:** The agent may move to a neighboring cell based on `prob_migrate`.
- **Kill Tumor Cell:** The agent attempts to kill a tumor cell in its neighborhood with `prob_kill`.
This method encapsulates the agent's decision-making and simulates its role in attacking tumor cells and moving through the environment.


In [ ]:

    def step(self):
        if not self.alive:
            return
        
        self.eat(5)
        if self.random.random() < self.prob_death: 
            self.alive = False
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
        if self.random.random() < self.prob_migrate:
            self.migrate()
        if self.random.random() < self.prob_kill:
            self.kill_tumor_cell()
    
    """
    Moves the agent to a random neighboring cell if the new cell is empty.
    """



## Migrate
Moves the M1 macrophage to a random neighboring cell if the new cell is empty.
Simulates the natural migration of macrophages as they patrol the environment for tumor cells.
Migration allows macrophages to interact with a wider area of the model, increasing their chances of encountering and attacking tumor cells.


In [ ]:

    def migrate(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        # Filter only empty positions
        empty_positions = [pos for pos in possible_steps if self.model.grid.is_cell_empty(pos)]

        #Pick an empty position if there are any
        if len(empty_positions) > 0:
            new_position = self.random.choice(empty_positions)
            self.model.grid.move_agent(self, new_position)

    """
    Kills a neighboring tumor cell if one exists.
    Reduces the killing capacity of the agent by 1.
    """
    def kill_tumor_cell(self):
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        tumor_cells = [cell for cell in neighbors if isinstance(cell, Tumor_cells)]
        if tumor_cells:
            #print("Attempting to kill TUMOR")
            target = self.random.choice(tumor_cells)
            target.apoptosis()#set_death_prob(1, "val") # Before TC.apoptosis() was called raising NoneType Error
            #print("Kill!")
            self.killing_capacity -= 1